In [13]:
# Data processing
import pandas as pd
from surprise import Reader
from surprise import Dataset, SVD, accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

In [14]:
matrix = pd.read_pickle("collaborative_filtering/data/item_matrix.pkl")
matrix.head()

userID,0,4,5,7,14,20,31,33,40,46,...,199956,199969,199973,199974,199975,199976,199980,199988,199990,199996
songID,,,,,,,,,,,,,,,,,,,,,
2263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
2726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
3785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8063,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
print(matrix.min().min(), matrix.max().max())

1.0 5.0


In [16]:
def get_data(matrix):
    matrix_filled = matrix.fillna(0)
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(matrix_filled.stack().reset_index(), reader)
    return data

In [17]:
# 5 fold cross-validation
algo = SVD()
cross_validate(algo, get_data(matrix), cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0840  1.0830  1.0831  1.0838  1.0850  1.0838  0.0007  
MAE (testset)     1.0345  1.0341  1.0343  1.0348  1.0351  1.0346  0.0003  
Fit time          53.37   44.44   41.27   40.13   38.43   43.53   5.30    
Test time         8.16    5.52    5.72    5.36    5.56    6.06    1.05    


{'test_rmse': array([1.08404778, 1.08302315, 1.08310106, 1.0837823 , 1.08496254]),
 'test_mae': array([1.0345276 , 1.03414374, 1.03432244, 1.03475599, 1.03514151]),
 'fit_time': (53.36607766151428,
  44.44030523300171,
  41.2745521068573,
  40.126304626464844,
  38.42705702781677),
 'test_time': (8.158060073852539,
  5.517755508422852,
  5.721208333969116,
  5.3621978759765625,
  5.564457178115845)}

In [18]:
data = get_data(matrix)
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=0.25)
# We'll use the famous SVD algorithm.
algo = SVD()
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)
# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 1.0836


1.0835660261344078